In [ ]:
# playground for GNN data processing

import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch as th


import os

In [ ]:
# extract filenames from npy_GNN folder

def get_filenames(directory):
    filenames = []
    for file in os.listdir(f"../npy_GNN/{directory}"):
        if file[-4:] == ".npy":
            filenames.append(file)
    filenames.sort()
    return filenames

names = get_filenames("2024-05-15_11:57:51_dynamic")
print(names)

# create a second list with the same filenames but without everything after the first underscore

def get_filenames_no_time(directory):
    filenames = []
    for file in os.listdir(f"../npy_GNN/{directory}"):
        if file[-4:] == ".npy":
            filenames.append(file)
    filenames.sort()
    for i in range(len(filenames)):
        filenames[i] = filenames[i].split("_")[0] + ".npy"
    return filenames

names_no_time = get_filenames_no_time("2024-05-15_11:57:51_dynamic")
types = len(set(names_no_time))

samples = len(names) // types
print(types, samples)

In [ ]:
# create a graph 
#load displacement data 
high_res_displacement = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[samples*3]}")
low_res_displacement = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[0]}")

high_res_velocity = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[samples*4]}")
low_res_velocity = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[samples*5]}")

#node features are just high res displacement and velocity horizontally stacked
node_features = np.hstack((high_res_displacement, high_res_velocity))
print(node_features.shape)
edge_index = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[samples*2]}")[:, :2].T
edge_attr = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[samples*2]}")[:, 2]

print(edge_index)
print(edge_attr.shape)

y = high_res_displacement - low_res_displacement

print(y.shape)

data = Data(x=th.tensor(node_features, dtype=th.float32), edge_index=th.tensor(edge_index, dtype=th.long), edge_attr=th.tensor(edge_attr, dtype=th.float32), y=th.tensor(y, dtype=th.float32))

# create a function to transform the data into a list of Data objects

def create_data_list(directory):
    names = get_filenames(directory)
    names_no_time = get_filenames_no_time(directory)
    types = len(set(names_no_time))
    samples = len(names) // types
    data_list = []
    for i in range(samples):
        high_res_displacement = np.load(f"../npy_GNN/{directory}/{names[samples*3+i]}")
        low_res_displacement = np.load(f"../npy_GNN/{directory}/{names[i]}")
        high_res_velocity = np.load(f"../npy_GNN/{directory}/{names[samples*4+i]}")
        node_features = np.hstack((high_res_displacement, high_res_velocity))
        edge_index = np.load(f"../npy_GNN/{directory}/{names[samples*2+i]}")[:, :2].T
        edge_attr = np.load(f"../npy_GNN/{directory}/{names[samples*2+i]}")[:, 2]
        y = high_res_displacement - low_res_displacement
        data = Data(x=th.tensor(node_features, dtype=th.float32), edge_index=th.tensor(edge_index, dtype=th.long), edge_attr=th.tensor(edge_attr, dtype=th.float32), y=th.tensor(y, dtype=th.float32))
        data_list.append(data)
    return data_list

data_list = create_data_list("2024-05-15_16:08:01_dynamic")
print(len(data_list))

In [ ]:
# build a GNN model that takes as input a Data object and outputs a prediction of shape (250, 3)
from torch.functional import F

class Net(th.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(6, 16)
        self.conv_mid = GCNConv(16, 16)
        self.conv2 = GCNConv(16, 3)
        self.repeat = 10 

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        for i in range(self.repeat):
            x = self.conv_mid(x, edge_index, edge_attr)
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return x

model = Net()

# train the model

from torch_geometric.loader import DataLoader

device = th.device('cuda' if th.cuda.is_available() else 'cpu')
model = model.to(device)
data_list = [data.to(device) for data in data_list]


loader = DataLoader(data_list, batch_size=32, shuffle=True)

model.train()

optimizer = th.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(2000):
    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.mse_loss(out, data.y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, Loss {loss.item()}")
  



In [ ]:
# test the model

model.eval()

for data in loader:
    out = model(data)
    print(out.shape)
    print(data.y.shape)
    print(F.mse_loss(out, data.y).item())
